## Advanced options to customize Counterfactual Explanations

Here we discuss a few ways to change DiCE's behavior. 

* Train a custom ML model 
* Changing feature weights that decide relative importance of features in perturbation
* Trading off between proximity and diversity goals
* Selecting the features to change

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

### Loading dataset

We use "adult" income dataset from UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/adult). For demonstration purposes, we transform the data as detailed in **dice_ml.utils.helpers** module. 

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

### 1. Training a custom ML model

Below, we build an Artificial Neural Network based on Keras Tensorflow framework.

In [5]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [6]:
sess = tf.InteractiveSession()

train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']

ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l1(0.001), activation=tf.nn.relu))
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=100, verbose=0, class_weight={0:1,1:2})


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
# provide the trained ML model to DiCE's model object
m = dice_ml.Model(model=ann_model) 

### Generate diverse counterfactuals

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Instructions for updating:
Use tf.cast instead.
Diverse Counterfactuals found! total time taken: 00 min 17 sec


In [11]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.017367



Diverse Counterfactual set after sparsity correction (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,73.0,Private,Masters,Single,White-Collar,White,Female,45.0,0.514
1,28.0,Private,Doctorate,Married,Service,White,Female,36.0,0.835
2,31.0,Self-Employed,Prof-school,Married,Professional,White,Female,45.0,0.904
3,43.0,Private,HS-grad,Married,Service,White,Female,66.0,0.677


### 2. Changing feature weights

It may be the case that some features are harder to change than others (e.g., education level is harder to change than working hours per week). DiCE allows input of relative difficulty in changing a feature through specifying *feature weights*. A higher feature weight means that the feature is harder to change than others. For instance, one way is to use the mean absolute deviation from the median as a measure of relative difficulty of changing a continuous feature.

Median Absolute Deviation (MAD) of a continuous feature conveys the variability of the feature, and is more robust than standard deviation as is less affected by outliers and non-normality. The inverse of MAD would then imply the ease of varying the feature and is hence used as feature weights in our optimization to reflect the difficulty of changing a continuous feature. By default, DiCE computes this internally and divides the distance between continuous features by the MAD of the feature's values in the training set. Let's see what their values are by computing them below:

In [12]:
# get MAD
mads = d.get_mads(normalized=True)

# create feature weights
feature_weights = {}
for feature in mads:
    feature_weights[feature] = round(1/mads[feature], 2)
print(feature_weights)

{'age': 7.3, 'hours_per_week': 24.5}


The above feature weights encode that changing *age* is approximately seven times more difficult than changing categorical variables, and changing *hours_per_week* is approximately three times more difficult than changing *age*. Of course, this may sound odd, since a person cannot change their age. In this case, what it's reflecting is that there is a higher diversity in age values than hours-per-week values. Below we show how to over-ride these weights to assign custom user-defined weights.

Now, let's try to assign equal weights to all features and see how it affects the counterfactual generation. DiCE allows this through *feature_weights* parameter.

In [13]:
# assigning equal weights
feature_weights = {'age': 1, 'hours_per_week': 1}

In [14]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        feature_weights=feature_weights)

Diverse Counterfactuals found! total time taken: 00 min 14 sec


In [15]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.017367



Diverse Counterfactual set after sparsity correction (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,59.0,Self-Employed,Doctorate,Married,Professional,White,Female,57.0,0.933
1,90.0,Private,HS-grad,Married,Service,White,Female,74.0,0.734
2,59.0,Government,Prof-school,Single,Service,White,Female,99.0,0.760
3,62.0,Private,HS-grad,Single,White-Collar,Other,Male,98.0,0.600


Note that we transform continuous features and one-hot-encode categorical features to fall between 0 and 1 in order to handle relative scale of features. However, this also means that the relative ease of changing continuous features is higher than categorical features when the total number of continuous features are very less compared to the total number of categories of all categorical variables combined. This is reflected in the above table where continuous features (*age* and *hours_per_week*) have been varied to reach their extreme values (*range of age: [17, 90]*; *range of hours_per_week: [1, 99]*) for most of the counterfactuals. This is the reason why the distances are divided by a scaling factor. Deviation from the median provides a robust measure of the variability of a feature’s values, and thus dividing by the MAD allows us to capture the relative prevalence of observing the feature at a particular value (see our [paper](https://arxiv.org/pdf/1905.07697.pdf) for more details).

### 3. Trading off between proximity and diversity goals

We acknowledge that not all counterfactual explanations may be feasible for a user. In general, counterfactuals closer to an individual's profile will be more feasible. Diversity is also important to help an individual choose between multiple possible options. DiCE allows tunable parameters *proximity_weight* (default: 0.5) and *diversity_weight* (default: 1.0) to handle proximity and diversity respectively. Below, we increse the proximity weight and see how the counterfactuals change.

In [16]:
# change proximity_weight from default value of 0.5 to 1.5
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        proximity_weight=1.5, diversity_weight=1.0) 

Diverse Counterfactuals found! total time taken: 00 min 25 sec


In [17]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.017367



Diverse Counterfactual set after sparsity correction (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,40.0,Private,Masters,Married,Service,White,Female,45.0,0.853
1,18.0,Self-Employed,Doctorate,Married,Service,White,Female,45.0,0.830
2,31.0,Private,HS-grad,Married,Professional,White,Female,45.0,0.645
3,22.0,Private,Prof-school,Married,Service,White,Female,55.0,0.781


As we see from above table, both continuous and categorical features are more closer to the original query instance and the counterfactuals are also less diverse than before.

### 4. Selecting the features to vary

While counterfactuals provide *actionable* alternative profiles to achieve a different outcome, we note that some of the generated explanations suggest changes in features that cannot be varied easily (such as age), or sensitive attributes like race or gender. Hence, DiCE allows feeding in a list of features that are allowed to vary through a *features_to_vary* parameter. 

In [18]:
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite", 
                                        features_to_vary=['workclass','education','occupation','hours_per_week'])

Diverse Counterfactuals found! total time taken: 00 min 24 sec


In [19]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.017367



Diverse Counterfactual set after sparsity correction (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Self-Employed,Doctorate,Single,Professional,White,Female,63.0,0.613
1,22.0,Self-Employed,Doctorate,Single,Service,White,Female,80.0,0.650
2,22.0,Self-Employed,Doctorate,Single,White-Collar,White,Female,91.0,0.779
3,22.0,Self-Employed,Masters,Single,Service,White,Female,99.0,0.678


The above counterfactual examples show the importance of educational level in the model's predictions. All the generated counterfactuals suggest getting an advanced degree for a higher income (Masters or Ph.D.).